In [1]:
import pandas as pd
df = pd.read_csv("mirrorPublications_featured.csv")
df

,Writing NFT,Author Homepage,Total Sold(ETH),Total Sold Numbers,Total Buyers,Price(ETH),nft_address,collection,fees,created_date,...,cleaned_title,processed_cleaned_text,cleaned_text_tokenized,cleaned_text_lemmatized,cleaned_text_POS,cleaned_text_sentiment,words_body,words_title,words_text,normalized_tfidf_sum
0,https://opensea.io/assets/optimism/0x96c200ec4...,https://mirror.xyz/0xe822ecac55a3a20bb4b24cdd8...,73.830,107,105,0.690,0x96c200ec4cca0bc57444cfee888cfba78a1ddbd8,lore-7-miberamaker-design-document,[2.5],2024-01-29,...,lore miberamaker design document,"{'tokens': ['lore', 'miberamaker', 'design', '...","['lore', 'miberamaker', 'design', 'document', ...","['lore', 'miberamaker', 'design', 'document', ...","[('lore', 'PROPN'), ('miberamaker', 'NOUN'), (...",0.266667,61,4,65,0.085601
1,https://opensea.io/assets/optimism/0x7988434e1...,https://mirror.xyz/0xe822ecac55a3a20bb4b24cdd8...,52.080,124,122,0.420,0x7988434e1469d35fa5f442e649de45d47c3df23c,lore-6-miberamaker-design-document,[2.5],2024-01-22,...,lore miberamaker design document,"{'tokens': ['lore', 'miberamaker', 'design', '...","['lore', 'miberamaker', 'design', 'document', ...","['lore', 'miberamaker', 'design', 'document', ...","[('lore', 'PROPN'), ('miberamaker', 'NOUN'), (...",0.200397,156,4,160,0.052217
2,https://opensea.io/assets/optimism/0x093312465...,https://mirror.xyz/0xe822ecac55a3a20bb4b24cdd8...,23.982,571,497,0.042,0x093312465e593c65b1f00e0ab8c5885634b2aa83,berafi-bong-bears-x-nftfi,"[2.5, 10.0]",2023-02-23,...,berafi bong bears nftfi,"{'tokens': ['berafi', 'bong', 'bears', 'nftfi'...","['berafi', 'bong', 'bears', 'nftfi', 'baby', '...","['berafi', 'bong', 'bear', 'nftfi', 'baby', 'b...","[('berafi', 'PROPN'), ('bong', 'PROPN'), ('bea...",0.154107,1412,4,1416,0.010898
3,https://opensea.io/assets/optimism/0x901c2436d...,https://mirror.xyz/0xff5ae9b0a7522736299d797d8...,20.000,10000,9855,0.002,0x901c2436d9d57cb8c0fc47d10e30c0c5be943dcb,announcing-the-results-of-retropgf-2,"[2.5, 10.0]",2023-03-30,...,announcing results retropgf,"{'tokens': ['announcing', 'results', 'retropgf...","['announcing', 'results', 'retropgf', 'rewardi...","['announce', 'result', 'retropgf', 'reward', '...","[('announcing', 'VERB'), ('results', 'NOUN'), ...",0.123364,369,3,372,0.022258
4,https://opensea.io/assets/optimism/0xe53f95be1...,https://mirror.xyz/0x000002c34bae6dd7bec72acba...,20.000,10000,9144,0.002,0xe53f95be16ee1d2a6a6afb3eedbb9969d337dea7,syncswap-era-testnet-is-now-live,"[2.5, 10.0]",2023-03-01,...,syncswap era testnet live,"{'tokens': ['syncswap', 'era', 'testnet', 'liv...","['syncswap', 'era', 'testnet', 'live', 'wait',...","['syncswap', 'era', 'testnet', 'live', 'wait',...","[('syncswap', 'PROPN'), ('era', 'NOUN'), ('tes...",0.195397,373,4,377,0.024740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26874,https://opensea.io/assets/optimism/0xe6aa6aaf3...,https://mirror.xyz/0x75aaf4a296c7e115c5757f936...,0.000,14,14,0.000,0xe6aa6aaf3ba199a09042cd00fb0052fa9d6f33c0,superseed-joins-the-superchain,[2.5],2024-10-15,...,superseed joins superchain,"{'tokens': ['superseed', 'joins', 'superchain'...","['superseed', 'joins', 'superchain', 'today', ...","['superseed', 'join', 'superchain', 'today', '...","[('superseed', 'NOUN'), ('joins', 'VERB'), ('s...",0.155277,253,3,256,0.029418
26875,https://opensea.io/assets/optimism/0xe82941552...,https://mirror.xyz/0x8073639b11994c549eda58fc3...,0.000,2,2,0.000,0xe829415523b4d89361116dde50a5eb841b60c71a,gr15-how-to-donate-to-gitcoin-grants,[2.5],2023-06-27,...,gr15 donate gitcoin grants,"{'tokens': ['gr15', 'donate', 'gitcoin', 'gran...","['gr15', 'donate', 'gitcoin', 'grants', 'guide...","['gr15', 'donate', 'gitcoin', 'grant', 'guide'...","[('gr15', 'NOUN'), ('donate', 'VERB'), ('gitco...",0.159822,1259,4,1263,0.010867
26876,https://opensea.io/assets/optimism/0x0eedbed03...,https://mirror.xyz/0x179b847833532783f84667ece...,0.000,1,1,0.000,0x0eedbed0325298e6b42d4b1904e42ecc8f40041e,good-proje

In [2]:
import spacy
from tqdm import tqdm

nlp = spacy.load("en_core_web_sm")
texts = []

for doc in tqdm(df['cleaned_text'], desc="Estrazione lemmi..."):
    spacy_doc = nlp(doc)
    lemmas = [token.lemma_ for token in spacy_doc if not token.is_stop and not token.is_punct]
    texts.append(lemmas)

Estrazione lemmi...: 100%|████████████████| 26879/26879 [25:09<00:00, 17.81it/s]


In [4]:
from gensim.models import Phrases

bigram = Phrases(texts, min_count=15)

texts_with_bigrams = []

for doc in tqdm(texts, desc="Aggiunta bigrammi..."):
    bigrams = bigram[doc]
    texts_with_bigrams.append(bigrams)

Aggiunta bigrammi...: 100%|█████████████| 26879/26879 [00:07<00:00, 3805.43it/s]


In [5]:
import gensim
from gensim import corpora
from gensim.models import LdaModel

dictionary = corpora.Dictionary(texts_with_bigrams) #creo il dizionario

print("Grandezza dizionario prima della rimozione:")
print(len(dictionary))

#rimuovo le parole che appaiono in meno di 6 documenti o in più del 30% dei documenti
dictionary.filter_extremes(no_below=6, no_above=0.3)

print("Grandezza dizionario dopo la rimozione:")
print(len(dictionary))

#corpus è una matrice di frequenza (bag of words) creata dal dizionario
corpus = [dictionary.doc2bow(text) for text in texts]

#training finale su 7 topics, che sembra essere il numero ottimale
num_topics = 7
chunksize = 1000
passes = 50
iterations = 600
eval_every = None

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

for topic in top_topics:
    print()
    print (topic[1], [w[1] for w in topic[0]])

Grandezza dizionario prima della rimozione:
328950
Grandezza dizionario dopo la rimozione:
37988
Average topic coherence: -1.1421.

-0.8904706890079639 ['people', 'think', 'thing', 'know', 'want', 'good', 'year', 'come', 'go', 'look', 'day', 'get', 'lot', 'find', 'feel', 'say', 'try', 'let', 'long', 'right']

-1.0364316402382014 ['blockchain', 'web3', 'nerk', 'crypto', 'system', 'datum', 'platform', 'technology', 'dao', 'social', 'world', 'model', 'ecosystem', 'ai', 'digital', 'value', 'application', 'decentralize', 'chain', 'product']

-1.062137486160156 ['game', 'reward', 'launch', 'mint', 'earn', 'player', 'team', 'point', 'platform', 'event', 'discord', 'airdrop', 'join', 'ecosystem', 'holder', 'twitter', 'week', 'web3', 'nfts', 'experience']

-1.2094783704821515 ['contract', 'address', 'wallet', 'account', 'datum', 'function', 'proposal', 'smart', 'mint', 'key', 'ethereum', 'chain', 'transaction', 'protocol', 'code', 'vote', 'app', 'optimism', 'eth', 'public']

-1.2144891361175718

In [27]:
#creo un dizionario di nomi di topic
topic_names = {
    0: 'T1: Human Thoughts & General Trends',
    1: 'T2: Web3 & Digital Ecosystems',
    2: 'T3: Gaming, Rewards & Community',
    3: 'T4: Smart Contracts & Proposals',
    4: 'T5: DeFi & Market Strategies',
    5: 'T6: Art, Creativity & Digital Culture',
    6: 'T7: Wallets, Transactions & Protocols'
}

#ottengo il topic dominante per ciascun documento e restituisco il nome di quel topic
def get_dominant_topic_name(text, model, dictionary):
    #converto il testo in bag of words
    bow = dictionary.doc2bow(text.split())  # Assicurati che text sia una stringa di parole
    #ottengo la distribuzione dei topic per il documento
    topic_probs = model.get_document_topics(bow)
    #trovo e restituisco il nome del topic con la probabilità più alta
    dominant_topic = max(topic_probs, key=lambda x: x[1])[0]
    return topic_names[dominant_topic]

#applico la funzione al df per creare la colonna topic
df['topic'] = df['cleaned_text'].apply(lambda x: get_dominant_topic_name(x, model, dictionary))

topic_counts = df['topic'].value_counts()
topic_counts

topic
T7: Wallets, Transactions & Protocols    5720
T2: Web3 & Digital Ecosystems            4958
T3: Gaming, Rewards & Community          4772
T6: Art, Creativity & Digital Culture    3877
T1: Human Thoughts & General Trends      2989
T5: DeFi & Market Strategies             2811
T4: Smart Contracts & Proposals          1752
Name: count, dtype: int64

In [31]:
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.save_html(lda_display, 'lda_visualization.html')
pyLDAvis.display(lda_display)

In [33]:
df.to_csv('mirrorPublications_withTopics.csv', index=False)

In [35]:
df

,Writing NFT,Author Homepage,Total Sold(ETH),Total Sold Numbers,Total Buyers,Price(ETH),nft_address,collection,fees,created_date,...,processed_cleaned_text,cleaned_text_tokenized,cleaned_text_lemmatized,cleaned_text_POS,cleaned_text_sentiment,words_body,words_title,words_text,normalized_tfidf_sum,topic
0,https://opensea.io/assets/optimism/0x96c200ec4...,https://mirror.xyz/0xe822ecac55a3a20bb4b24cdd8...,73.830,107,105,0.690,0x96c200ec4cca0bc57444cfee888cfba78a1ddbd8,lore-7-miberamaker-design-document,[2.5],2024-01-29,...,"{'tokens': ['lore', 'miberamaker', 'design', '...","['lore', 'miberamaker', 'design', 'document', ...","['lore', 'miberamaker', 'design', 'document', ...","[('lore', 'PROPN'), ('miberamaker', 'NOUN'), (...",0.266667,61,4,65,0.085601,T5: DeFi & Market Strategies
1,https://opensea.io/assets/optimism/0x7988434e1...,https://mirror.xyz/0xe822ecac55a3a20bb4b24cdd8...,52.080,124,122,0.420,0x7988434e1469d35fa5f442e649de45d47c3df23c,lore-6-miberamaker-design-document,[2.5],2024-01-22,...,"{'tokens': ['lore', 'miberamaker', 'design', '...","['lore', 'miberamaker', 'design', 'document', ...","['lore', 'miberamaker', 'design', 'document', ...","[('lore', 'PROPN'), ('miberamaker', 'NOUN'), (...",0.200397,156,4,160,0.052217,T5: DeFi & Market Strategies
2,https://opensea.io/assets/optimism/0x093312465...,https://mirror.xyz/0xe822ecac55a3a20bb4b24cdd8...,23.982,571,497,0.042,0x093312465e593c65b1f00e0ab8c5885634b2aa83,berafi-bong-bears-x-nftfi,"[2.5, 10.0]",2023-02-23,...,"{'tokens': ['berafi', 'bong', 'bears', 'nftfi'...","['berafi', 'bong', 'bears', 'nftfi', 'baby', '...","['berafi', 'bong', 'bear', 'nftfi', 'baby', 'b...","[('berafi', 'PROPN'), ('bong', 'PROPN'), ('bea...",0.154107,1412,4,1416,0.010898,T1: Human Thoughts & General Trends
3,https://opensea.io/assets/optimism/0x901c2436d...,https://mirror.xyz/0xff5ae9b0a7522736299d797d8...,20.000,10000,9855,0.002,0x901c2436d9d57cb8c0fc47d10e30c0c5be943dcb,announcing-the-results-of-retropgf-2,"[2.5, 10.0]",2023-03-30,...,"{'tokens': ['announcing', 'results', 'retropgf...","['announcing', 'results', 'retropgf', 'rewardi...","['announce', 'result', 'retropgf', 'reward', '...","[('announcing', 'VERB'), ('results', 'NOUN'), ...",0.123364,369,3,372,0.022258,T4: Smart Contracts & Proposals
4,https://opensea.io/assets/optimism/0xe53f95be1...,https://mirror.xyz/0x000002c34bae6dd7bec72acba...,20.000,10000,9144,0.002,0xe53f95be16ee1d2a6a6afb3eedbb9969d337dea7,syncswap-era-testnet-is-now-live,"[2.5, 10.0]",2023-03-01,...,"{'tokens': ['syncswap', 'era', 'testnet', 'liv...","['syncswap', 'era', 'testnet', 'live', 'wait',...","['syncswap', 'era', 'testnet', 'live', 'wait',...","[('syncswap', 'PROPN'), ('era', 'NOUN'), ('tes...",0.195397,373,4,377,0.024740,T1: Human Thoughts & General Trends
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26874,https://opensea.io/assets/optimism/0xe6aa6aaf3...,https://mirror.xyz/0x75aaf4a296c7e115c5757f936...,0.000,14,14,0.000,0xe6aa6aaf3ba199a09042cd00fb0052fa9d6f33c0,superseed-joins-the-superchain,[2.5],2024-10-15,...,"{'tokens': ['superseed', 'joins', 'superchain'...","['superseed', 'joins', 'superchain', 'today', ...","['superseed', 'join', 'superchain', 'today', '...","[('superseed', 'NOUN'), ('joins', 'VERB'), ('s...",0.155277,253,3,256,0.029418,"T7: Wallets, Transactions & Protocols"
26875,https://opensea.io/assets/optimism/0xe82941552...,https://mirror.xyz/0x8073639b11994c549eda58fc3...,0.000,2,2,0.000,0xe829415523b4d89361116dde50a5eb841b60c71a,gr15-how-to-donate-to-gitcoin-grants,[2.5],2023-06-27,...,"{'tokens': ['gr15', 'donate', 'gitcoin', 'gran...","['gr15', 'donate', 'gitcoin', 'grants', 'guide...","['gr15', 'donate', 'gitcoin', 'grant', 'guide'...","[('gr15', 'NOUN'), ('donate', 'VERB'), ('gitco...",0.159822,1259,4,1263,0.010867,"T6: Art, Creativity & Digital Culture"
26876,https://opensea.io/assets/optimism/0x0eedbed03...,https://mirror.xyz/0x179b847833532783f84667ece...,0.000,1,1,0.000,0x0eedbed0325298e